In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import json

In [2]:
class NepaliEcommerceBot:
    def __init__(self):
        # Common Romanized Nepali e-commerce queries and responses
        self.training_data = {
            "queries": [
                "mero order kahile aaucha",
                "delivery charge kati ho",
                "return policy k cha",
                "payment kahan garne",
                "online payment garna milcha",
                "cash on delivery cha",
                "size chart hernu milcha",
                "discount cha",
                "free shipping cha",
                "order cancel garna milcha"
            ],
            "responses": [
                "Tapai ko order 3-5 din bhitra deliver hunecha.",
                "Delivery charge Rs. 100 valley bhitra ra Rs. 200 valley bahira cha.",
                "7 din bhitra return garna milcha, product unopened hunu parcha.",
                "Hamro ma cash on delivery ra online payment dubai cha.",
                "Ho, hamro ma credit card, debit card ra e-banking ko facility cha.",
                "Ho, cash on delivery ko facility cha.",
                "Size chart product ko description ma cha, hernus.",
                "Ahile Rs. 1000 bhanda mathi ko order ma 10% discount cha.",
                "Rs. 2000 bhanda mathi ko order ma free shipping cha.",
                "Order confirm bhayeko 24 ghanta bhitra cancel garna milcha."
            ]
        }

        # Initialize TF-IDF vectorizer
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.vectorizer.fit_transform(self.training_data["queries"])

    def preprocess_text(self, text):
        """Basic text preprocessing for Romanized Nepali"""
        return text.lower().strip()

    def get_response(self, user_query):
        """Get most relevant response for user query"""
        processed_query = self.preprocess_text(user_query)
        query_vector = self.vectorizer.transform([processed_query])

        # Calculate similarity scores
        similarities = cosine_similarity(query_vector, self.tfidf_matrix)[0]

        # Get most similar response
        best_match_index = np.argmax(similarities)
        confidence_score = similarities[best_match_index]

        if confidence_score < 0.2:
            return "Maaf garnuhos, ma tapai ko prashna bujhina. Kripaya arko tarika le sodhnus."

        return self.training_data["responses"][best_match_index]

    def add_training_data(self, query, response):
        """Add new query-response pair to training data"""
        self.training_data["queries"].append(query)
        self.training_data["responses"].append(response)
        self.tfidf_matrix = self.vectorizer.fit_transform(self.training_data["queries"])

    def save_model(self, filename):
        """Save training data to JSON file"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.training_data, f, ensure_ascii=False, indent=4)

    def load_model(self, filename):
        """Load training data from JSON file"""
        with open(filename, 'r', encoding='utf-8') as f:
            self.training_data = json.load(f)
            self.tfidf_matrix = self.vectorizer.fit_transform(self.training_data["queries"])


In [7]:
# Example usage
bot = NepaliEcommerceBot()

# Adding more training data
bot.add_training_data(
    "shipping time kati lagcha",
    "Valley bhitra 2-3 din ra valley bahira 4-5 din lagcha."
)

# Test the chatbot
test_queries = [
    "mero order ko barema bhannu hos",
    "delivery ko paisa kasto xa?",
    "maile return garna milxa ki mildaina?",
    "payment ko lagi kasto subidha xa?",
    "return method k k cha",
    "shipping option kasto cha"
]

for query in test_queries:
    print(f"Query: {query}")
    print(f"Response: {bot.get_response(query)}\n")

Query: mero order ko barema bhannu hos
Response: Tapai ko order 3-5 din bhitra deliver hunecha.

Query: delivery ko paisa kasto xa?
Response: Ho, cash on delivery ko facility cha.

Query: maile return garna milxa ki mildaina?
Response: 7 din bhitra return garna milcha, product unopened hunu parcha.

Query: payment ko lagi kasto subidha xa?
Response: Hamro ma cash on delivery ra online payment dubai cha.

Query: return method k k cha
Response: 7 din bhitra return garna milcha, product unopened hunu parcha.

Query: shipping option kasto cha
Response: Rs. 2000 bhanda mathi ko order ma free shipping cha.

